In [39]:
from myUtils import pickle_load
verbose = False  # True prints more output

In [40]:
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
filename_pickled_df_symbols_close = "df_symbols_close"  # pickled filename

In [41]:
# load df_symbols_close
df = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_orig = df.copy()  # backup df
df = df.tail(252)  # keep only the last 252 days of data
symbols_before_dropna = df.columns

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close


In [42]:
df.shape

(252, 1917)

In [43]:
df = df.dropna(axis=1,how='any')  # drop symbols (i.e. columns) with NaN
symbols_after_dropna = df.columns
symbols_dropped = symbols_before_dropna.difference(symbols_after_dropna).tolist()
# print(f'symbols with NaN dropped from dataframe :{symbols_dropped}')
print(f'{len(symbols_dropped)} symbols have NaN in their close price, and were dropped from the dataframe.')
print(f'These symbols were dropped:\n{symbols_dropped}')

49 symbols have NaN in their close price, and were dropped from the dataframe.
These symbols were dropped:
['ACT', 'BF-A', 'BLCO', 'BRCC', 'BRZE', 'BXSL', 'CEG', 'CMTG', 'CRDO', 'CRGY', 'CWAN', 'DH', 'EE', 'ESAB', 'ESMT', 'FLNC', 'FRSH', 'FYBR', 'GBTG', 'GFS', 'GOOG', 'GTLB', 'HCP', 'HKD', 'HLN', 'IHS', 'INFA', 'IOT', 'KD', 'LTH', 'NU', 'OLPX', 'ONON', 'PFHC', 'RIVN', 'RTLR', 'SLVM', 'SMCI', 'SMR', 'SNAP', 'SNOW', 'SRAD', 'STER', 'TCN', 'TOST', 'TWKS', 'UDMY', 'VALE', 'WE']


In [44]:
df.shape

(252, 1868)

In [45]:
df.columns

Index(['A', 'AA', 'AAL', 'AAON', 'AAP', 'AAPL', 'AAWW', 'ABB', 'ABBV', 'ABC',
       ...
       'ZI', 'ZIM', 'ZION', 'ZLAB', 'ZM', 'ZNH', 'ZS', 'ZTO', 'ZTS', 'ZWS'],
      dtype='object', length=1868)

In [46]:
df.to_csv('C:/Users/ping/Desktop/df_close.csv')

In [47]:
from myUtils import symb_perf_stats_vectorized_v1
symbols, period_yr, drawdown, UI, max_drawdown, \
        returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = symb_perf_stats_vectorized_v1(df)

In [48]:
print(f'drawdown.shape: {drawdown.shape}')
print(f'type(drawdown): {type(drawdown)}')

drawdown.shape: (252, 1868)
type(drawdown): <class 'numpy.ndarray'>


In [49]:
def moving_window(arr, window):
    # reference: http://www.rigtorp.se/2011/01/01/rolling-statistics-numpy.html
    """Create moving-window of numpy.ndarray from a numpy.ndarray or pandas.Series

    Args:
        arr(float): a numpy.ndarray or pandas.Series
        window(int): width of moving-window

    Return:
        numpy.ndarray with shape [len(arr) - window + 1, window]
            where each row has elements of the moving_window,
            and number of elements in each row is the window size (i.e. columns)
    """

    import numpy as np

    shape = arr.shape[:-1] + (arr.shape[-1] - window + 1, window)
    strides = arr.strides + (arr.strides[-1],)

    return np.lib.stride_tricks.as_strided(arr, shape=shape, strides=strides)

In [50]:
def UI_MW(arr_drawdown, window=20):
    """Calculate Ulcer-Index on moving-window applied to
       arr_drawdown(numpy.ndarray)

    Args:
        arr_drawdown(numpy array): drawdown from function symb_perf_stats or
                                   from ulcer_index.py
        window(int): moving-window size, default = 20

    Return:
        UI_MW(numpy.ndarray): Ulcer-Index on moving-window
        applied to arr_drawdown
    """

    import numpy as np

    # apply moving window to arr_drawdown, result is numpy ndarray
    DD_MW = moving_window(arr_drawdown, window)
    DD_MW_square = np.square(DD_MW)
    sum_DD_MW_square = np.sum(DD_MW_square, axis=1)  # sum each row
    UI_MW = np.sqrt(sum_DD_MW_square / window)
    UI_MW = nan_pad(len(arr_drawdown), UI_MW)  # pad with leading NaN

    return UI_MW

In [51]:
def UI_MW_15(arr_drawdown):
    """Calculate Ulcer-Index on moving-window applied to
       arr_drawdown(numpy.ndarray)

    Args:
        arr_drawdown(numpy array): drawdown from function symb_perf_stats or
                                   from ulcer_index.py

    Return:
        UI_MW(numpy.ndarray): Ulcer-Index on moving-window
        applied to arr_drawdown
    """

    import numpy as np

    # apply moving window to arr_drawdown, result is numpy ndarray
    window = 15
    DD_MW = moving_window(arr_drawdown, window)
    DD_MW_square = np.square(DD_MW)
    sum_DD_MW_square = np.sum(DD_MW_square, axis=1)  # sum each row
    UI_MW = np.sqrt(sum_DD_MW_square / window)
    UI_MW = nan_pad(len(arr_drawdown), UI_MW)  # pad with leading NaN

    return UI_MW

In [52]:
def UI_MW_30(arr_drawdown):
    """Calculate Ulcer-Index on moving-window applied to
       arr_drawdown(numpy.ndarray)

    Args:
        arr_drawdown(numpy array): drawdown from function symb_perf_stats or
                                   from ulcer_index.py

    Return:
        UI_MW(numpy.ndarray): Ulcer-Index on moving-window
        applied to arr_drawdown
    """

    import numpy as np

    # apply moving window to arr_drawdown, result is numpy ndarray
    window = 30
    DD_MW = moving_window(arr_drawdown, window)
    DD_MW_square = np.square(DD_MW)
    sum_DD_MW_square = np.sum(DD_MW_square, axis=1)  # sum each row
    UI_MW = np.sqrt(sum_DD_MW_square / window)
    UI_MW = nan_pad(len(arr_drawdown), UI_MW)  # pad with leading NaN

    return UI_MW

In [53]:
def nan_pad(arr_len, target_arr):
    """Pad target_arr with leading numpy.nan to length arr_len.

    Args:
        arr_len(int): length of return array
        target_arr(numpy.ndarray): array to be padded with leading numpy.nan

    Return:
        padded_target_arr(numpy.ndarray): target_arr padded to length arr_len
    """

    import numpy as np

    pad_len = arr_len - len(target_arr)
    pad_arr = np.empty(pad_len, dtype=float)
    pad_arr.fill(np.nan)
    padded_target_arr = np.concatenate((pad_arr, target_arr), axis=0)

    return padded_target_arr

In [54]:
import numpy as np
UIMW15 = np.apply_along_axis(UI_MW_15, 0, drawdown)
UIMW30 = np.apply_along_axis(UI_MW_30, 0, drawdown)
print(f'UIMW15.shape : {UIMW15.shape}')
print(f'UIMW30.shape : {UIMW30.shape}')

UIMW15.shape : (252, 1868)
UIMW30.shape : (252, 1868)


In [55]:
anr = abs(1 - UIMW30/UIMW15)  # absolute normalized ratio  
anr.shape

c:\Users\ping\MyDrive\py_files\python\py379\.venv\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


(252, 1868)

In [56]:
anr15 = anr[-15:, :]
anr15

array([[0.12453346, 0.06768343, 0.01789851, ..., 0.025568  , 0.00842149,
        0.00104848],
       [0.12899102, 0.06941397, 0.02974693, ..., 0.01655646, 0.00492237,
        0.00444972],
       [0.13808685, 0.06935886, 0.03425825, ..., 0.01479635, 0.00279604,
        0.00094188],
       ...,
       [0.0353628 , 0.05565929, 0.00244608, ..., 0.00191617, 0.06690209,
        0.00701941],
       [0.02906457, 0.04397743, 0.00983587, ..., 0.00680259, 0.06855256,
        0.00530903],
       [0.01788422, 0.0350411 , 0.02461877, ..., 0.00749654, 0.07347182,
        0.00180595]])

In [57]:
fMask = np.where((anr15 < .0005), True, False)  # filtered masked 2d np.array
idx_filtr_anr15 = np.column_stack(np.where(fMask))
# len(idx_filtr_anr15)

In [58]:
idx_syms = idx_filtr_anr15[:,1].tolist()
idx_syms = list(set(idx_syms))
idx_syms.sort()

In [59]:
fSyms = []
for idx in idx_syms:
  fSyms.append(df.columns[idx])
len(fSyms)

90

In [ ]:
['RPRX', 'SNA', 'VRTX']

In [71]:
import yfinance as yf
from myUtils import yf_print_symbol_data, yf_candlestick
# plot symbols
plot_days = '252d'
for symbol in fSyms[80:95]:
    # yf.Ticker OHLC are adjusted correctly
    df = yf.Ticker(symbol).history(period=plot_days)
    df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)  # drop Div. and Stock Splits columns
    df.columns = ['open', 'high', 'low', 'close', 'volume']  # rename columns Open ... Volume to open ... volume
    cache_plot = yf_candlestick(symbol, df, plot_chart=True)